In [1]:
from netCDF4 import Dataset
import wrf
import xarray as xr
import numpy as np

import cartopy.crs as crs


from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cartopy.feature as cfe

import pandas as pd


import matplotlib.pyplot as plt



from cartopy.feature import NaturalEarthFeature

import cartopy.io.shapereader as shpr

import cartopy.io.img_tiles as cimgt

from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)

from matplotlib.cm import get_cmap
from wrf import getvar, interplevel, to_np, get_basemap, latlon_coords
import imageio

In [2]:
stamen_terrain = cimgt.Stamen('terrain-background')

states = NaturalEarthFeature(category="cultural", scale="10m",
                                 facecolor="none",
                                 name="admin_1_states_provinces_shp")
stt = NaturalEarthFeature(category='cultural', 
    name='admin_0_boundary_lines_land',
    scale='10m',facecolor='none')
stt_prv = states
roads = NaturalEarthFeature(category='cultural',
                                     name='roads',
                                     scale='10m',
                                     facecolor='none')

# Presión
## Función para hacer las animaciones de la presión

In [29]:
# Abrir el dataset
def gif_presion(fichero,dominio):
    ncfile = Dataset(fichero)
    for i in range(0,168):
        
        # Leer la presión a nivel del mar en eltiempo t 
        slp = getvar(ncfile, "slp",timeidx=i)

        #Para suavizar la presión a nivel del mar
        smooth_slp = smooth2d(slp, 3, cenweight=4)

        # Para leer la latitud y la longitud
        lats, lons = latlon_coords(slp)

        #Para leer la proyección en la que se ha hecho
        cart_proj = get_cartopy(slp)

        # Crear la figura para mostrarlo
        fig = plt.figure(figsize=(12,6))

        #Poner los GeoAxes en la proyección usada por el modelo 
        ax = plt.axes(projection=cart_proj)

        #Para mostrar las lineas de costa
        ax.coastlines('50m', linewidth=0.8)

        #Para hacer el contorno de los puntos que tienen misma presión
        plt.contour(to_np(lons), to_np(lats), to_np(smooth_slp),levels=[j for j in range(1000, 1031)] , colors="black",
                    transform=crs.PlateCarree(),linewidth=0.1)
        #Para rellenar lo puntos que tienen la misma presión
        contorno_slp= plt.contourf(to_np(lons), to_np(lats), to_np(smooth_slp),
                     transform=crs.PlateCarree(),
                    levels=[j for j in range(1000, 1031)],
                     cmap=get_cmap("jet"))

        # Add a color bar
        plt.colorbar(ax=ax, shrink=.98)

        # Set the map bounds
        ax.set_xlim(cartopy_xlim(smooth_slp))
        ax.set_ylim(cartopy_ylim(smooth_slp))

        # Add the gridlines
        ax.gridlines(color="gray", linestyle="dotted")

        ax.add_feature(states, linewidth=.9, edgecolor="black")

        #Mostrar las carreteras 
        ax.add_feature(roads,linewidth=.4, edgecolor="black")

        plt.title("Presion al nivel del mar (hPa)\n Día {} de Mayo Hora {}:00:00".format(int(i/24)+11,(i-int(i/24)*24)))
        plt.savefig('figs/d{}/presion/d{}_presion_{}.png'.format(dominio, dominio,i))
        #Para que no se muestren las imágenes
        plt.close(fig)



    with imageio.get_writer('figs/d{}/d{}_presion.gif'.format(dominio,dominio), mode='I') as writer:
        for i in range(0,168):
            for j in range(0,3):
                filename = 'figs/d{}/presion/d{}_presion_{}.png'.format(dominio,dominio,i)
                image = imageio.imread(filename)
                writer.append_data(image)

## Dominio 1

In [30]:
gif_presion('wrfout_d01_2021-05-11_00:00:00',"01")

<img src="figs/d01/d01_presion.gif" />

## Dominio 2

In [7]:
gif_presion('wrfout_d02_2021-05-11_00:00:00',"02")

<img src="figs/d02/d02_presion.gif" />

## Dominio 3


In [11]:
gif_presion('wrfout_d03_2021-05-11_00:00:00',"03")

/home/fernan/miniconda3/envs/ncl_stable/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1478: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


<img src="figs/d03/d03_presion.gif" />

# Mostrar los 3 dominios

In [25]:
# Abrir el dataset
def gif_presion_3_doms(fichero):
    ncfile_01 = Dataset(fichero[0])
    ncfile_02 = Dataset(fichero[1])
    ncfile_03 = Dataset(fichero[2])

    for i in range(0,168):
        # Leer la presión a nivel del mar en eltiempo t 
        slp_01 = getvar(ncfile_01, "slp",timeidx=i)
        slp_02 = getvar(ncfile_02, "slp",timeidx=i)
        slp_03 = getvar(ncfile_03, "slp",timeidx=i)
        
        #Para suavizar la presión a nivel del mar
        smooth_slp_1 = smooth2d(slp_01, 3, cenweight=4)
        smooth_slp_2 = smooth2d(slp_02, 3, cenweight=4)
        smooth_slp_3 = smooth2d(slp_03, 3, cenweight=4)

        #Para leer la proyección en la que se ha hecho
        cart_proj = get_cartopy(slp_03)

        # Crear la figura para mostrarlo
        fig = plt.figure(figsize=(15, 6))
        
        niveles = [j for j in range(1000, 1031)]
        #Ciudad de Madrid
        # Para leer la latitud y la longitud
        lats, lons = latlon_coords(slp_03)
        
        ax = fig.add_subplot(1, 5, (1,4), projection=cart_proj)
        # Set the map bounds
        ax.set_xlim(cartopy_xlim(smooth_slp_3))
        ax.set_ylim(cartopy_ylim(smooth_slp_3))
        
        ax.set_title('Presión al nivel del mar en Madrid \n Día {} Hora {}:00:00'.format(int(i/24)+11,(i-int(i/24)*24)), fontsize=16)
        ax.coastlines('50m', linewidth=0.8)
    
        #Para hacer el contorno de los puntos que tienen misma presión
        plt.contour(to_np(lons), to_np(lats), to_np(smooth_slp_3),levels=niveles, 
                    colors="black",
                    transform=crs.PlateCarree(),linewidth=0.1)
        #Para rellenar lo puntos que tienen la misma presión
        contorno_slp= plt.contourf(to_np(lons), to_np(lats), to_np(smooth_slp_3),
                     transform=crs.PlateCarree(),
                    levels=niveles,
                     cmap=get_cmap("jet"))

        # Add a color bar
        plt.colorbar(contorno_slp,ax=ax, shrink=.98)
        ax.add_feature(stt, linewidth=0.2, edgecolor='black')
        
        #Mostrar las carreteras 
        ax.add_feature(roads,linewidth=.4, edgecolor="black")

        # Add the gridlines
        ax.gridlines(color="gray", linestyle="dotted")

        
        
        
        # Comunidad de Madrid
        lats, lons = latlon_coords(slp_02)
        
        ax_2 = fig.add_subplot(2, 5, 5,projection=cart_proj)
        
        ax_2.set_xlim(cartopy_xlim(smooth_slp_2))
        ax_2.set_ylim(cartopy_ylim(smooth_slp_2))
        
        ax_2.set_title('Comunidad de Madrid', fontsize=8)
        
        ax_2.coastlines('50m', linewidth=0.8)

        
        plt.contour(to_np(lons), to_np(lats), to_np(smooth_slp_2),levels=niveles,
                    colors="black",
                    transform=crs.PlateCarree(),linewidth=0.1)
        
        #Para rellenar lo puntos que tienen la misma presión
        contorno_slp= plt.contourf(to_np(lons), to_np(lats), to_np(smooth_slp_2),
                     transform=crs.PlateCarree(),
                    levels=niveles,
                     cmap=get_cmap("jet"))
        
        ax_2.add_feature(stt,     linewidth=0.5, edgecolor='black')
        ax_2.add_feature(stt_prv, linewidth=0.2, edgecolor='black')
        ax_2.add_feature(roads,linewidth=.4, edgecolor="black")
        
        
        
        
        #España
        lats, lons = latlon_coords(slp_01)
        
        ax_1 = fig.add_subplot(2, 5, 10, projection=cart_proj)
        
        ax_1.set_title('España', fontsize=8)
        ax_1.coastlines('50m', linewidth=0.8)
        
        plt.contour(to_np(lons), to_np(lats), to_np(smooth_slp_1),levels=niveles, colors="black",
                    transform=crs.PlateCarree(),linewidth=0.1)
        #Para rellenar lo puntos que tienen la misma presión
        contorno_slp= plt.contourf(to_np(lons), to_np(lats), to_np(smooth_slp_1),
                     transform=crs.PlateCarree(),
                    levels=niveles,
                     cmap=get_cmap("jet"))
        
        ax_1.add_feature(stt,     linewidth=0.5, edgecolor='black', zorder=0)
        ax_1.add_feature(stt_prv, linewidth=0.2, edgecolor='black', zorder=5)
        ax_1.add_feature(roads,linewidth=.4, edgecolor="black")

        

        
        plt.savefig('figs/presion/presion_{}.png'.format(i))
        #Para que no se muestren las imágenes
        plt.close(fig)

    with imageio.get_writer('figs/presion.gif', mode='I') as writer:
            for i in range(0,56):
                for j in range(0,3):
                    filename = 'figs/presion/presion_{}.png'.format(i)
                    image = imageio.imread(filename)
                    writer.append_data(image)    

In [26]:
gif_presion_3_doms(['wrfout_d01_2021-05-11_00:00:00',
                        'wrfout_d02_2021-05-11_00:00:00',
                        'wrfout_d03_2021-05-11_00:00:00'])

<img src="figs/pression.gif"/>